In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader,Dataset
from torchvision import transforms
from torchinfo import summary
from torchvision.io import read_image
from torchvision.utils import make_grid,save_image
import matplotlib.pyplot as plt
import torchvision.transforms.functional as TF
import torch.optim as optim
import numpy as np
import os
from tqdm import tqdm
import numpy
import itertools
from PIL import Image
# import cv2
#watch -n 1 nvidia-smi --id=3

In [2]:
torch.cuda.device_count()
cuda = torch.cuda.is_available()
print(f'cuda: {cuda}')

cuda: True


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.set_device(0)
# Batch_size = 2
# Learning_rate = 2e-4
# # num_epochs = 100
# # labda_cycle = 10
# # lambda_identity = 1
# load_model = False
# save_model = True
checkpoint_gen_PM = '/kaggle/input/weights/checkpoints/genpm.pth.tar'
checkpoint_gen_MP = '/kaggle/input/weights/checkpoints/genmp.pth.tar'
checkpoint_critic_P = '/kaggle/input/weights/checkpoints/criticp.pth.tar'
checkpoint_critic_M = '/kaggle/input/weights/checkpoints/criticm.pth.tar'
# root_photo = '/home/sanketwr/project1/practise /datasets/photo_jpg'
# root_monet = '/home/sanketwr/project1/practise /datasets/monet_jpg'
# transform = transforms.Compose([
#     # transforms.ToTensor()
#     # transforms.Resize(size=(256, 256)),
#     transforms.RandomHorizontalFlip(p = 0.5),
#     # transforms.RandomVerticalFlip(p =0.5),
#     transforms.ToTensor(),
#     transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))
# ])


In [4]:
# ## Discriminator
# class Block(nn.Module):
#     def __init__(self,in_channels,out_channels,stride):
#         super().__init__()
#         self.conv = nn.Sequential(
#             nn.Conv2d(in_channels=in_channels,out_channels = out_channels,kernel_size=4,stride = stride,padding= 1,bias = True,padding_mode='reflect'),
#             nn.InstanceNorm2d(out_channels),
#             nn.LeakyReLU(0.2)
#         )
#     def forward(self,x):
#         return self.conv(x)
# class Discriminator(nn.Module):
#     def __init__(self,in_channels=3,features = [64,128,256,512]):
#         super().__init__()
#         self.initial = nn.Sequential(
#             nn.Conv2d(
#             in_channels,
#             features[0],
#             kernel_size=4,
#             stride=2,
#             padding=1,
#             padding_mode='reflect'
#             ),
#             nn.LeakyReLU(0.2)
#         )

#         layers = []
#         in_channels = features[0]
#         for feature in features[1:]:
#             layers.append(Block(in_channels,feature,stride=1 if feature == features[-1] else 2))
#             in_channels = feature

#         layers.append(nn.Conv2d(in_channels,out_channels=3,kernel_size=4,stride=1,padding = 1,padding_mode='reflect'))
#         self.model = nn.Sequential(*layers)

#     def forward(self,x):
#         x = self.initial(x)
#         return torch.sigmoid(self.model(x))
    
# def test():
#     x = torch.randn((10,3,256,256))
#     model = Discriminator()
#     preds = model(x)
#     # print(summary(model))
#     print(preds.shape)

# if __name__ == "__main__":
#     test()
    


class Discriminator(nn.Module):
    def __init__(self, in_channels):
        super(Discriminator, self).__init__()
        
        self.model = nn.Sequential(
            # why normalize=False?
            *self.block(in_channels, 64, normalize=False), # 3*256*256 -> 64*128*128 
            *self.block(64, 128),  # 64*128*128 -> 128*64*64
            *self.block(128, 256), # 128*64*64 -> 256*32*32
            *self.block(256, 512), # 256*32*32 -> 512*16*16
            
            # Why padding first then convolution?
            nn.ZeroPad2d((1,0,1,0)), # padding left and top   512*16*16 -> 512*17*17
            nn.Conv2d(512, 1, 4, padding=1) # 512*17*17 -> 1*16*16
        )
        
        self.scale_factor = 16
    
    @staticmethod
    def block(in_channels, out_channels, normalize=True):
        layers = [nn.Conv2d(in_channels, out_channels, 4, stride=2, padding=1)]
        if normalize:
            layers.append(nn.InstanceNorm2d(out_channels))
        layers.append(nn.LeakyReLU(0.2, inplace=True))
        
        return layers
        
    def forward(self, x):
        return self.model(x)

In [5]:
class ConvBlock(nn.Module):
    def __init__(self,in_channels,out_channels,down = True, use_act = True,**kwargs):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels,out_channels,padding_mode='reflect',**kwargs)
            if down
            else nn.ConvTranspose2d(in_channels,out_channels,**kwargs),
            nn.InstanceNorm2d(out_channels),
            nn.ReLU(inplace=True) if use_act else nn.Identity()
        )
    def forward(self,x):
        return self.conv(x)
class ResidualBlock(nn.Module):
    def __init__(self,channels):
        super().__init__()
        self.block = nn.Sequential(
            # ConvBlock(channels,channels,kernel_size = 3, padding =1),
            # # nn.InstanceNorm2d(channels),
            # ConvBlock(channels,channels,use_act = False,kernel_size = 3,padding =1),
            # # nn.InstanceNorm2d(channels),

            nn.ReflectionPad2d(1), # padding, keep the image size constant after next conv2d
            nn.Conv2d(channels, channels, kernel_size = 3),
            nn.InstanceNorm2d(channels),
            nn.ReLU(inplace=True),
            nn.ReflectionPad2d(1),
            nn.Conv2d(channels, channels, kernel_size=3),
            nn.InstanceNorm2d(channels)
        )
    def forward(self,x):
        return x + self.block(x)
    
class Generator(nn.Module):
    def __init__(self,img_channels,num_features = 64,num_residuals = 9):
        super().__init__()
        self.inital = nn.Sequential(
            nn.Conv2d(img_channels,num_features,kernel_size=7,stride = 1, padding=3,padding_mode='reflect'),
            nn.InstanceNorm2d(num_features),
            nn.ReLU(inplace = True),
        )
        self.down_blocks = nn.Sequential(
                ConvBlock(num_features,num_features*2,kernel_size = 3,stride = 2,padding =1),
                # nn.InstanceNorm2d(num_features*2),
                ConvBlock(num_features*2,num_features*4,kernel_size = 3,stride = 2,padding =1),
                # nn.InstanceNorm2d(num_features*4),
                
        )
        self.residual_blocks = nn.Sequential(
            *[ResidualBlock(num_features*4) for _ in range(num_residuals)]
        )
        self.up_block = nn.Sequential(
            
                ConvBlock(num_features*4,num_features*2,down = False,kernel_size = 3,stride = 2,padding =1,output_padding = 1),
                ConvBlock(num_features*2,num_features*1,down = False,kernel_size = 3,stride = 2,padding =1,output_padding = 1),                 
            
        )
        self.last = nn.Conv2d(num_features*1,img_channels,kernel_size=7,stride = 1,padding=3,padding_mode='reflect')

    def forward(self,x):
        x = self.inital(x)
        # for layer in self.down_blocks:
        x = self.down_blocks(x)
        x = self.residual_blocks(x)
        # for layer in self.up_block(x):
        x = self.up_block(x)
        return torch.tanh(self.last(x))
    
def test():
    img_channels =3
    img_size = 256
    x = torch.randn((2,img_channels,img_size,img_size))
    gen = Generator(img_channels,9)
    # print(summary(gen))
    print(gen(x).shape) 
if __name__ == "__main__":
    test()

torch.Size([2, 3, 256, 256])


In [6]:

class customData(Dataset):
    def __init__(self,root_A,root_B,mode='train',transform = None):
        self.root_A = root_A #monet
        self.root_B = root_B #photo
        self.transform = transform
        # self.image_A = os.listdir(self.root_A)[:4000]
        # self.image_B = os.listdir(self.root_B)
        if mode == 'train':
            self.files_M = [os.path.join(root_A, name) for name in sorted(os.listdir(root_A))[:250]]
            self.files_P = [os.path.join(root_B, name) for name in sorted(os.listdir(root_B))[:250]]
        elif mode == 'test':
            self.files_M = [os.path.join(root_A, name) for name in sorted(os.listdir(root_A))[250:]]
            self.files_P = [os.path.join(root_B, name) for name in sorted(os.listdir(root_B))[250:301]]
        

        # self.length_dataset = max(len(self.image_A),len(self.image_B)) #7038,300
        # self.len_A = len(self.image_A)
        # self.len_B = len(self.image_B)

    def __len__(self):
        return len(self.files_M)
    
    def __getitem__(self,idx):

        # image_path_A = os.path.join(self.root_A,self.image_A[idx % self.len_A])
        # image_path_B = os.path.join(self.root_B,self.image_B[idx % self.len_B])
        # image_A = read_image(image_path_A)
        # image_B =  read_image(image_path_B)
        file_M = self.files_M[idx]
        file_P = self.files_P[idx]
        img_M = Image.open(file_M)
        img_P = Image.open(file_P)
        
        if self.transform is not None:
            img_M = self.transform(img_M)
            img_P = self.transform(img_P)
        
        return img_M, img_P


In [7]:
# criterion_GAN = nn.MSELoss().to(device)
# criterion_cycle = nn.L1Loss().to(device)
# criterion_identity = nn.L1Loss().to(device)
# G_PM = Generator(img_channels = 3,num_residuals=9).to(device)
# D_M  = Discriminator(in_channels=3).to(device=device)
# G_MP = Generator(img_channels = 3,num_residuals=9).to(device)
# D_P = Discriminator(in_channels=3).to(device=device)
# c_Data_image = customData(root_monet,root_photo,transform=transform)
lr =  0.0005
b1 = 0.5
b2 = 0.996

# optimizer_G = optim.Adam(itertools.chain(G_PM.parameters(),G_MP.parameters()),lr = lr,betas=(b1,b2))
# optimizer_D_M = optim.Adam(D_M.parameters(),lr = lr,betas=(b1,b2))
# optimizer_D_P = optim.Adam(D_P.parameters(),lr = lr,betas=(b1,b2))

## learning rate scheduler

n_epochs = 120
decay_epoch = 20


# lambda_func = lambda epoch: 1 - max(0, epoch-decay_epoch)/(n_epochs-decay_epoch)

# lr_scheduler_G = torch.optim.lr_scheduler.LambdaLR(optimizer_G, lr_lambda=lambda_func)
# lr_scheduler_D_M = torch.optim.lr_scheduler.LambdaLR(optimizer_D_M, lr_lambda=lambda_func)
# lr_scheduler_D_P = torch.optim.lr_scheduler.LambdaLR(optimizer_D_P, lr_lambda=lambda_func)

# # c_Data_monet = customData(root_monet)
# print(c_Data_image.__len__())
# data_loader = DataLoader(c_Data_image,batch_size=Batch_size,shuffle=True)
# # monet_loader = DataLoader(c_Data_monet,batch_size=batch_size,shuffle=True)
# for i, [image_A,image_B] in enumerate(data_loader):
#     if i == 1:
#         break
#     print(image_A.shape,image_B.shape)
#     # images = TF.to_pil_image(images).convert('RGB')
#     image_batch_np_A = make_grid(image_A,normalize=True).numpy()
#     image_batch_np_B = make_grid(image_B,normalize=True).numpy()
#     image_batch_np_A = np.transpose(image_batch_np_A,(1,2,0))
#     image_batch_np_B = np.transpose(image_batch_np_B,(1,2,0))

#     plt.figure(figsize=(1, 1))
#     plt.axis("off")
#     plt.title("Sample Images")
#     plt.imshow(image_batch_np_A)
#     # plt.show()
#     plt.figure(figsize=(1, 1))
#     plt.axis("off")
#     plt.title("Sample Images")
#     plt.imshow(image_batch_np_B)
#     plt.show()

batch_size = 5

# trainloader = DataLoader(
#     customData(root_monet,root_photo, mode='train', transform=transform),
#     batch_size = batch_size,
#     shuffle = True,
    
# )

# testloader = DataLoader(
#     customData(root_monet,root_photo, mode='test', transform=transform),
#     batch_size = batch_size,
#     shuffle = False,
# )

Tensor = (torch.cuda.FloatTensor) if device else torch.Tensor
# Tensor = Tensor.to(device)
def sample_images(real_M, real_P, figside=1.5):
    assert real_M.size() == real_P.size(), 'The image size for two domains must be the same'
    
    G_PM.eval()
    G_MP.eval()
    
    real_M = real_M.type(Tensor)#.to(device)
    fake_P = G_MP(real_M).detach()
    real_P = real_P.type(Tensor)#.to(device)
    fake_M = G_PM(real_P).detach()
    
    nrows = real_M.size(0)
    real_P = make_grid(real_P, nrow=nrows, normalize=True)
    fake_M = make_grid(fake_M, nrow=nrows, normalize=True)
    real_M = make_grid(real_M, nrow=nrows, normalize=True)
    fake_P = make_grid(fake_P, nrow=nrows, normalize=True)
    
    image_grid = torch.cat((real_P, fake_M, real_M, fake_P), 1).cpu().permute(1, 2, 0)
    
    plt.figure(figsize=(figside*nrows, figside*4))
    plt.imshow(image_grid)
    plt.axis('off')
    plt.show()



In [8]:
# real_M, real_P = next(iter(testloader))
# sample_images(real_M, real_P)

In [9]:
def save_checkpoint(model,optimizer,filename = '/home/sanketwr/project1/practise /CV codes/checkpoints/cyclegan.pth.tar'):
    # print('=> Saving Checkpoint')
    checkpoint = {'model': model.state_dict(),
              'optimizer': optimizer.state_dict(),
              }
    torch.save(checkpoint, filename)

def load_checkpoint(checkpoint_file,model,optimizer):
    print('=> Loading checkpoint')
    checkpoint = torch.load(checkpoint_file, map_location=device)
    model.load_state_dict(checkpoint['model'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    

In [10]:
load_model = True
def sample_images_(real_M, real_P, G_PM,G_MP,figside=1.5):
    assert real_M.size() == real_P.size(), 'The image size for two domains must be the same'
    
    G_PM.eval()
    G_MP.eval()
    
    real_M = real_M.type(Tensor)#.to(device)
    fake_P = G_MP(real_M).detach()
    real_P = real_P.type(Tensor)#.to(device)
    fake_M = G_PM(real_P).detach()
    
    nrows = real_M.size(0)
    real_P = make_grid(real_P, nrow=nrows, normalize=True)
    fake_M = make_grid(fake_M, nrow=nrows, normalize=True)
    real_M = make_grid(real_M, nrow=nrows, normalize=True)
    fake_P = make_grid(fake_P, nrow=nrows, normalize=True)
    
    image_grid = torch.cat((real_P, fake_M, real_M, fake_P), 1).cpu().permute(1, 2, 0)
    
    plt.figure(figsize=(figside*nrows, figside*4))
    plt.imshow(image_grid)
    plt.axis('off')
    plt.show()



def train_fn(D_M,D_P,G_MP,G_PM,trainloader,testloader,optimizer_D_M,optimizer_D_P,optimizer_G,criterion_GAN,criterion_cycle,criterion_identity,d_scaler,g_scaler,epoch,lr_scheduler_G,lr_scheduler_D_M,lr_scheduler_D_P):
    loop = tqdm(trainloader,leave = True)
    for idx , (real_M,real_P) in enumerate(loop):

        real_M, real_P = real_M.type(Tensor).to(device), real_P.type(Tensor).to(device)
        
        # groud truth
        out_shape = [real_M.size(0), 1, real_M.size(2)//16, real_P.size(3)//16]
        valid = torch.ones(out_shape).type(Tensor).to(device)
        fake = torch.zeros(out_shape).type(Tensor).to(device)
        """Train Generators"""
        # set to training mode in the begining, beacause sample_images will set it to eval mode
        G_MP.train()
        G_PM.train()
        fake_P = G_MP(real_M)
        fake_M = G_PM(real_P)
        # trainig generators :
        # with torch.cuda.amp.autocast():
        # identity loss
        loss_id_M = criterion_identity(fake_P, real_M)
        loss_id_P = criterion_identity(fake_M, real_P)
        loss_identity = (loss_id_M + loss_id_P) / 2
        
        # GAN loss, train G to make D think it's true
        loss_GAN_MP = criterion_GAN(D_P(fake_P), valid) 
        loss_GAN_PM = criterion_GAN(D_M(fake_M), valid)
        loss_GAN = (loss_GAN_MP + loss_GAN_PM) / 2
        
        # cycle loss
        recov_M = G_PM(fake_P)
        recov_P = G_MP(fake_M)
        loss_cycle_M = criterion_cycle(recov_M, real_M)
        loss_cycle_P = criterion_cycle(recov_P, real_P)
        loss_cycle = (loss_cycle_M + loss_cycle_P) / 2
        
        # G totol loss
        loss_G = 5.0*loss_identity + loss_GAN + 10.0*loss_cycle

        optimizer_G.zero_grad()
        loss_G.backward()
        optimizer_G.step()
            
        # with torch.cuda.amp.autocast():
        loss_real = criterion_GAN(D_M(real_M), valid)
        loss_fake = criterion_GAN(D_M(fake_M.detach()), fake)
        loss_D_M = (loss_real + loss_fake) / 2
            
            
        """Train Discriminator B"""
            
            
        loss_real = criterion_GAN(D_P(real_P), valid)
        loss_fake = criterion_GAN(D_P(fake_P.detach()), fake)
        loss_D_P = (loss_real + loss_fake) / 2
            



        optimizer_D_M.zero_grad()
        loss_D_M.backward()
        optimizer_D_M.step()

        optimizer_D_P.zero_grad()
        # d_scaler.scale(loss_D_P).backward()
        # d_scaler.step(optimizer_D_P)
        # d_scaler.update()
        loss_D_P.backward()
        optimizer_D_P.step()
        

    if (epoch+1) % 10 == 0:
        test_real_M, test_real_P = next(iter(testloader))
        sample_images_(test_real_M, test_real_P,G_PM,G_MP)


    loss_D = (loss_D_M + loss_D_P) / 2
    print(f'[Epoch {epoch+1}/{n_epochs}]')
    print(f'[G loss: {loss_G.item()} | identity: {loss_identity.item()} GAN: {loss_GAN.item()} cycle: {loss_cycle.item()}]')
    print(f'[D loss: {loss_D.item()} | D_A: {loss_D_M.item()} D_B: {loss_D_P.item()}]')    

            
def main(l):
    # load_model = True
    # disc_P = Discriminator(in_channels=3).to(device=device)
    # disc_M = Disc/riminator(in_channels=3).to(device=device)

    # gen_PM = Generator(img_channels = 3,num_residuals=9).to(device) 
    # gen_MP = Generator(img_channels = 3,num_residuals=9).to(device)

    
    G_PM = Generator(img_channels = 3,num_residuals=9).to(device)
    D_M  = Discriminator(in_channels=3).to(device=device)
    G_MP = Generator(img_channels = 3,num_residuals=9).to(device)
    D_P = Discriminator(in_channels=3).to(device=device)
    
    optimizer_G = optim.Adam(itertools.chain(G_MP.parameters(),G_PM.parameters()),lr = lr,betas=(b1,b2))
    optimizer_D_M = optim.Adam(D_M.parameters(),lr = lr,betas=(b1,b2))
    optimizer_D_P = optim.Adam(D_P.parameters(),lr = lr,betas=(b1,b2))
        
    lambda_func = lambda epoch: 1 - max(0, epoch-decay_epoch)/(n_epochs-decay_epoch)
    lr_scheduler_G = torch.optim.lr_scheduler.LambdaLR(optimizer_G, lr_lambda=lambda_func)
    lr_scheduler_D_M = torch.optim.lr_scheduler.LambdaLR(optimizer_D_M, lr_lambda=lambda_func)
    lr_scheduler_D_P = torch.optim.lr_scheduler.LambdaLR(optimizer_D_P, lr_lambda=lambda_func)


    criterion_GAN = nn.MSELoss().to(device)
    criterion_cycle = nn.L1Loss().to(device)
    criterion_identity = nn.L1Loss().to(device)

    if load_model:
        load_checkpoint(
            checkpoint_critic_M,D_M,optimizer_D_M,Learning_rate,
        )
        load_checkpoint(
            checkpoint_critic_P,D_P,optimizer_D_P,Learning_rate,
        )
        load_checkpoint(
            checkpoint_gen_MP,G_MP,optimizer_G,Learning_rate,
        )
        load_checkpoint(
            checkpoint_gen_PM,G_PM,optimizer_G,Learning_rate,
        )
    batch_size = 2

    trainloader = DataLoader(
        customData(root_monet,root_photo, mode='train', transform=transform),
        batch_size = batch_size,
        shuffle = True,
        
    )

    testloader = DataLoader(
        customData(root_monet,root_photo, mode='test', transform=transform),
        batch_size = batch_size,
        shuffle = False,
    )

    g_scaler = torch.cuda.amp.GradScaler()
    d_scaler = torch.cuda.amp.GradScaler()


    for epoch in range(n_epochs):
        train_fn(D_M,D_P,G_MP,G_PM,trainloader,testloader,optimizer_D_M,optimizer_D_P,optimizer_G,criterion_GAN,criterion_cycle,criterion_identity,d_scaler,g_scaler,epoch,lr_scheduler_G,lr_scheduler_D_M,lr_scheduler_D_P)
        lr_scheduler_G.step()
        lr_scheduler_D_M.step()
        lr_scheduler_D_P.step()
        if save_model:
            print('saving checkpoints')
            save_checkpoint(G_MP,optimizer_G,filename=checkpoint_gen_MP)
            save_checkpoint(G_PM,optimizer_G,filename=checkpoint_gen_PM)
            save_checkpoint(D_M,optimizer_D_M,filename=checkpoint_critic_M)
            save_checkpoint(D_P,optimizer_D_P,filename=checkpoint_critic_P)
            
if __name__ == "__main__":
    l = 0
#     main(l)           

    

In [11]:
# root_photo = "/kaggle/input/gan-getting-started"
root_photo ="/kaggle/input/gan-getting-started/photo_jpg"# os.path.join(root_photo, 'photo_jpg')
files = [os.path.join(root_photo, name) for name in os.listdir(root_photo)]
len(files)
save_dir = '../images'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
generate_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

to_image = transforms.ToPILImage()
G_PM = Generator(img_channels = 3,num_residuals=9).to(device)
D_M  = Discriminator(in_channels=3).to(device=device)
G_MP = Generator(img_channels = 3,num_residuals=9).to(device)
D_P = Discriminator(in_channels=3).to(device=device)
optimizer_G = optim.Adam(itertools.chain(G_MP.parameters(),G_PM.parameters()),lr = lr,betas=(b1,b2))
optimizer_D_M = optim.Adam(D_M.parameters(),lr = lr,betas=(b1,b2))
optimizer_D_P = optim.Adam(D_P.parameters(),lr = lr,betas=(b1,b2))

load_checkpoint(
            checkpoint_critic_M,D_M,optimizer_D_M,
)
load_checkpoint(
    checkpoint_critic_P,D_P,optimizer_D_P,
)
load_checkpoint(
    checkpoint_gen_MP,G_MP,optimizer_G,
)
load_checkpoint(
    checkpoint_gen_PM,G_PM,optimizer_G,
)

G_PM.eval()
batch_size = 5
for i in range(0, len(files), batch_size):
    # read images
    imgs = []
    for j in range(i, min(len(files), i+batch_size)):
        img = Image.open(files[j])
        img = generate_transforms(img)
        imgs.append(img)
    imgs = torch.stack(imgs, 0).type(Tensor)
    
    # generate
    fake_imgs = G_PM(imgs).detach().cpu()
    
    # save
    for j in range(fake_imgs.size(0)):
        img = fake_imgs[j].squeeze().permute(1, 2, 0)
        img_arr = img.numpy()
        img_arr = (img_arr - np.min(img_arr)) * 255 / (np.max(img_arr) - np.min(img_arr))
        img_arr = img_arr.astype(np.uint8)
        
        img = to_image(img_arr)
        _, name = os.path.split(files[i+j])
        img.save(os.path.join(save_dir, name))

=> Loading checkpoint
=> Loading checkpoint
=> Loading checkpoint
=> Loading checkpoint


In [12]:
import shutil
shutil.make_archive("/kaggle/working/images", 'zip', "/kaggle/images")

'/kaggle/working/images.zip'